In [ ]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
import torchvision
import torchvision.transforms as transformations

transformation = transformations.ToTensor()
dataset_path = '/path/to/dataset'
mnist_train_set = torchvision.datasets.MNIST(root=dataset_path, train=True, transform=transformation, download=True)
mnist_test_set = torchvision.datasets.MNIST(root=dataset_path, train=False, transform=transformation, download=False)

mnist_validation_set = Subset(mnist_train_set, torch.arange(10000))
mnist_training_set = Subset(mnist_train_set, torch.arange(10000, len(mnist_train_set)))

batch_size = 64
training_loader = DataLoader(mnist_training_set, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(mnist_validation_set, batch_size=batch_size, shuffle=False)
testing_loader = DataLoader(mnist_test_set, batch_size=batch_size, shuffle=False)

class ConvNet(nn.Module):
    def __init__(self, kernel_size_second_layer):
        super(ConvNet, self).__init__()
        self.first_layer = nn.Sequential(
            nn.Conv2d(1, 4, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.second_layer = nn.Sequential(
            nn.Conv2d(4, 2, kernel_size=kernel_size_second_layer, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.additional_pooling = nn.MaxPool2d(kernel_size=2, stride=2)
        self.compute_output_size(kernel_size_second_layer)

    def compute_output_size(self, kernel_size_second_layer):
        output_dimension = (14 - kernel_size_second_layer) + 1
        output_dimension = output_dimension // 2
        output_dimension = output_dimension // 2
        num_features = 2 * output_dimension * output_dimension
        self.fully_connected = nn.Linear(num_features, 10)

    def forward(self, x):
        x = self.first_layer(x)
        first_layer_out_size = x.size()
        x = self.second_layer(x)
        second_layer_out_size = x.size()
        x = self.additional_pooling(x)
        additional_pooling_out_size = x.size()
        x = x.view(x.size(0), -1)
        final_out_size = x.size()
        x = self.fully_connected(x)
        return x, first_layer_out_size, second_layer_out_size, additional_pooling_out_size, final_out_size

def evaluate_on_test_set(model, loader):
    model.eval()
    total_correct = 0
    total_samples = 0
    with torch.no_grad():
        for imgs, labels in loader:
            outputs, *_ = model(imgs)
            _, predictions = torch.max(outputs.data, 1)
            total_samples += labels.size(0)
            total_correct += (predictions == labels).sum().item()
    accuracy = 100 * total_correct / total_samples
    print(f'Test Accuracy: {accuracy:.2f}%')

def train_and_validate(model, train_loader, valid_loader, test_loader, epochs=10, kernel_size=3):
    print(model)

    dummy_input = torch.randn(64, 1, 28, 28)
    _, first_layer_out_size, second_layer_out_size, additional_pooling_out_size, final_out_size = model(dummy_input)
    print(f'Output Size after First Layer: {first_layer_out_size}')
    print(f'Output Size after Second Layer: {second_layer_out_size}')
    print(f'Output Size after Additional Pooling: {additional_pooling_out_size}')
    print(f'Output Size before Fully Connected Layer: {final_out_size}')
    print(f"\nStarting training with kernel size {kernel_size} on second layer.")

    optimiser = optim.Adam(model.parameters(), lr=0.001)
    loss_function = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        model.train()
        total_loss = 0.0
        epoch_start = time.time()
        train_correct = 0
        train_total = 0

        for imgs, labels in train_loader:
            outputs = model(imgs)[0]
            loss = loss_function(outputs, labels)
            optimiser.zero_grad()
            loss.backward()
            optimiser.step()
            total_loss += loss.item()
            _, predicted_labels = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted_labels == labels).sum().item()

        training_accuracy = 100 * train_correct / train_total
        training_duration = time.time() - epoch_start

        model.eval()
        validation_loss = 0.0
        valid_correct = 0
        valid_total = 0
        with torch.no_grad():
            for imgs, labels in valid_loader:
                outputs = model(imgs)[0]
                loss = loss_function(outputs, labels)
                validation_loss += loss.item()
                _, predicted_labels = torch.max(outputs.data, 1)
                valid_total += labels.size(0)
                valid_correct += (predicted_labels == labels).sum().item()

        validation_accuracy = 100 * valid_correct / valid_total

        print(f'Epoch {epoch+1}, Training Loss: {total_loss/len(train_loader):.4f}, '
              f'Training Accuracy: {training_accuracy:.2f}%, '
              f'Validation Loss: {validation_loss/len(valid_loader):.4f}, '
              f'Validation Accuracy: {validation_accuracy:.2f}%, '
              f'Time: {training_duration:.2f}s')

    evaluate_on_test_set(model, test_loader)

model_instance_3 = ConvNet(kernel_size_second_layer=3)
train_and_validate(model_instance_3, training_loader, validation_loader, testing_loader, epochs=10, kernel_size=3)

model_instance_5 = ConvNet(kernel_size_second_layer=5)
train_and_validate(model_instance_5, training_loader, validation_loader, testing_loader, epochs=10, kernel_size=5)


ConvNet(
  (first_layer): Sequential(
    (0): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (second_layer): Sequential(
    (0): Conv2d(4, 2, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (additional_pooling): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fully_connected): Linear(in_features=18, out_features=10, bias=True)
)
Output Size after First Layer: torch.Size([64, 4, 14, 14])
Output Size after Second Layer: torch.Size([64, 2, 6, 6])
Output Size after Additional Pooling: torch.Size([64, 2, 3, 3])
Output Size before Fully Connected Layer: torch.Size([64, 18])

Starting training with kernel size 3 on second layer.
Epoch 1, Training Loss: 1.1884, Training Accuracy: 60.43%, Validation Loss: 0.6303, Validation Accuracy: 80.23%, Time: 10.